The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [169]:
%%capture

import sys
sys.path.append('..')
import mock_d2l_jax as d2l

# Multi-Branch Networks  (GoogLeNet)
:label:`sec_googlenet`

In 2014, *GoogLeNet*
won the ImageNet Challenge :cite:`Szegedy.Liu.Jia.ea.2015`, using a structure
that combined the strengths of NiN :cite:`Lin.Chen.Yan.2013`, repeated blocks :cite:`Simonyan.Zisserman.2014`,
and a cocktail of convolution kernels. It is arguably also the first network that exhibits a clear distinction among the stem, body, and head in a CNN. This design pattern has persisted ever since in the design of deep networks: the *stem* is given by the first 2-3 convolutions that operate on the image. They extract low-level features from the underlying images. This is followed by a *body* of convolutional blocks. Finally, the *head* maps the features obtained so far to the required classification, segmentation, detection, or tracking problem at hand.

The key contribution in GoogLeNet was the design of the network body. It solved the problem of selecting
convolution kernels in an ingenious way. While other works tried to identify which convolution, ranging from $1 \times 1$ to $11 \times 11$ would be best, it simply *concatenated* multi-branch convolutions.
In what follows we introduce a slightly simplified version of GoogLeNet. The simplifications are due to the fact that  tricks to stabilize training, in particular intermediate loss functions, are no longer needed due to the availability of improved training algorithms.

## (**Inception Blocks**)

The basic convolutional block in GoogLeNet is called an *Inception block*,
stemming from the meme "we need to go deeper" of the movie *Inception*.

![Structure of the Inception block.](http://d2l.ai/_images/inception.svg)
:label:`fig_inception`

As depicted in :numref:`fig_inception`,
the inception block consists of four parallel branches.
The first three branches use convolutional layers
with window sizes of $1\times 1$, $3\times 3$, and $5\times 5$
to extract information from different spatial sizes.
The middle two branches also add a $1\times 1$ convolution of the input
to reduce the number of channels, reducing the model's complexity.
The fourth branch uses a $3\times 3$ max-pooling layer,
followed by a $1\times 1$ convolutional layer
to change the number of channels.
The four branches all use appropriate padding to give the input and output the same height and width.
Finally, the outputs along each branch are concatenated
along the channel dimension and comprise the block's output.
The commonly-tuned hyperparameters of the Inception block
are the number of output channels per layer.


In [170]:
import jax
from jax import numpy as jnp, random, grad, vmap, jit
from flax import linen as nn
import optax
# from d2l import jax as d2l


class Inception(nn.Module):
    # `c1`--`c4` are the number of output channels for each branch
    c1: int
    c2: tuple
    c3: tuple
    c4: int
    
    def setup(self):
        # Branch 1
        self.b1_1 = nn.Conv(self.c1, kernel_size=(1, 1))
        # Branch 2
        self.b2_1 = nn.Conv(self.c2[0], kernel_size=(1, 1))
        self.b2_2 = nn.Conv(self.c2[1], kernel_size=(3, 3), padding='same')
        # Branch 3
        self.b3_1 = nn.Conv(self.c3[0], kernel_size=(1, 1))
        self.b3_2 = nn.Conv(self.c3[1], kernel_size=(5, 5), padding='same')
        # Branch 4
        self.b4_1 = lambda x: nn.max_pool(x, window_shape=(3, 3), strides=(1, 1), padding='same')
        self.b4_2 = nn.Conv(self.c4, kernel_size=(1, 1))

    def __call__(self, x):
        b1 = nn.relu(self.b1_1(x))
        b2 = nn.relu(self.b2_2(nn.relu(self.b2_1(x))))
        b3 = nn.relu(self.b3_2(nn.relu(self.b3_1(x))))
        b4 = nn.relu(self.b4_2(self.b4_1(x)))
        return jnp.concatenate((b1, b2, b3, b4), axis=-1)

To gain some intuition for why this network works so well,
consider the combination of the filters.
They explore the image in a variety of filter sizes.
This means that details at different extents
can be recognized efficiently by filters of different sizes.
At the same time, we can allocate different amounts of parameters
for different filters.


## [**GoogLeNet Model**]

As shown in :numref:`fig_inception_full`, GoogLeNet uses a stack of a total of 9 inception blocks, arranged into 3 groups with max-pooling in between,
and global average pooling in its head to generate its estimates.
Max-pooling between inception blocks reduces the dimensionality.
At its stem, the first module is similar to AlexNet and LeNet.

![The GoogLeNet architecture.](../img/inception-full.svg)
:label:`fig_inception_full`

We can now implement GoogLeNet piece by piece. Let's begin with the stem.
The first module uses a 64-channel $7\times 7$ convolutional layer.


In [171]:
class GoogleNet(d2l.Classifier):
    lr: float = 0.1
    num_classes: int = 10

In [172]:
@d2l.add_to_class(GoogleNet)
def b1(self):
    return [
        nn.Conv(64, kernel_size=(7, 7), strides=(2, 2), padding='same'), nn.relu,
        lambda x: nn.max_pool(x, window_shape=(3, 3), strides=(2, 2), padding='same')
    ]

The second module uses two convolutional layers:
first, a 64-channel $1\times 1$ convolutional layer,
followed by a $3\times 3$ convolutional layer that triples the number of channels. This corresponds to the second branch in the Inception block and concludes the design of the body. At this point we have 192 channels.


In [173]:
@d2l.add_to_class(GoogleNet)
def b2(self):
    return [
        nn.Conv(64, kernel_size=(3, 3)), nn.relu,
        nn.Conv(192, kernel_size=(3, 3), padding='same'), nn.relu,
        lambda x: nn.max_pool(x, window_shape=(3, 3), strides=(2, 2), padding='same')
    ]

The third module connects two complete Inception blocks in series.
The number of output channels of the first Inception block is
$64+128+32+32=256$. This amounts to 
a ratio of the number of output channels
among the four branches of $2:4:1:1$. Achieving this, we first reduce the input
dimensions by $\frac{1}{2}$ and by $\frac{1}{12}$ in the second and third branch respectively
to arrive at $96 = 192/2$ and $16 = 192/12$ channels respectively.

The number of output channels of the second Inception block
is increased to $128+192+96+64=480$, yielding a ratio of $128:192:96:64 = 4:6:3:2$. As before,
we need to reduce the number of intermediate dimensions in the second and third channel. A
scale of $\frac{1}{2}$ and $\frac{1}{8}$ respectively suffices, yielding $128$ and $32$ channels
respectively. This is captured by the arguments of the following `Inception` block constructors.


In [174]:
@d2l.add_to_class(GoogleNet)
def b3(self):
    return [Inception(64, (96, 128), (16, 32), 32),
            Inception(128, (128, 192), (32, 96), 64),
            lambda x: nn.max_pool(x, window_shape=(3, 3), strides=(2, 2), padding='same')]

The fourth module is more complicated.
It connects five Inception blocks in series,
and they have $192+208+48+64=512$, $160+224+64+64=512$,
$128+256+64+64=512$, $112+288+64+64=528$,
and $256+320+128+128=832$ output channels, respectively.
The number of channels assigned to these branches is similar
to that in the third module:
the second branch with the $3\times 3$ convolutional layer
outputs the largest number of channels,
followed by the first branch with only the $1\times 1$ convolutional layer,
the third branch with the $5\times 5$ convolutional layer,
and the fourth branch with the $3\times 3$ max-pooling layer.
The second and third branches will first reduce
the number of channels according to the ratio.
These ratios are slightly different in different Inception blocks.


In [175]:
@d2l.add_to_class(GoogleNet)
def b4(self):
    return [Inception(192, (96, 208), (16, 48), 64),
            Inception(160, (112, 224), (24, 64), 64),
            Inception(128, (128, 256), (24, 64), 64),
            Inception(112, (144, 288), (32, 64), 64),
            Inception(256, (160, 320), (32, 128), 128),
            lambda x: nn.max_pool(x, window_shape=(3, 3), strides=(2, 2), padding='same')]

The fifth module has two Inception blocks with $256+320+128+128=832$
and $384+384+128+128=1024$ output channels.
The number of channels assigned to each branch
is the same as that in the third and fourth modules,
but differs in specific values.
It should be noted that the fifth block is followed by the output layer.
This block uses the global average pooling layer
to change the height and width of each channel to 1, just as in NiN.
Finally, we turn the output into a two-dimensional array
followed by a fully connected layer
whose number of outputs is the number of label classes.


In [176]:
@d2l.add_to_class(GoogleNet)
def b5(self):
    return [Inception(256, (160, 320), (32, 128), 128),
            Inception(384, (192, 384), (48, 128), 128),
            lambda x: nn.avg_pool(x, (1, 1)),# TODO: nn.AdaptiveAvgPool2d((1,1)),
            d2l.flatten]

In [177]:
@d2l.add_to_class(GoogleNet)
def setup(self):
    self.net = nn.Sequential([*self.b1(), *self.b2(), *self.b3(), *self.b4(),
                              *self.b5(), nn.Dense(self.num_classes)])
    # self.net.apply(d2l.init_cnn)

The GoogLeNet model is computationally complex. Note the large number of
relatively arbitrary hyperparameters in terms of the number of channels chosen.
This work was done before scientists started using automatic tools to
optimize network designs.

For now the only modification we will carry out is to
[**reduce the input height and width from 224 to 96
to have a reasonable training time on Fashion-MNIST.**]
This simplifies the computation. Let's have a look at the
changes in the shape of the output between the various modules.


In [178]:
model = GoogleNet().layer_summary((1, 96, 96, 1))

                             GoogleNet Summary                              
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path               ┃ outputs              ┃ params                       ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs             │ float32[1,96,96,1]   │                              │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_0       │ float32[1,48,48,64]  │ bias: float32[64]            │
│                    │                      │ kernel: float32[7,7,1,64]    │
│                    │                      │                              │
│                    │                      │ 3,200 (12.8 KB)              │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11/b1_1 │ float32[1,6,6,192]   │ bias: float32[192]           │
│                    │                      │ kernel: float32[1,1,480,192] │
│                    │                      │                              │
│                    │                      │ 92,352 (369.4 KB)            │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11/b2_1 │ float32[1,6,6,96]    │ bias: float32[96]            │
│                    │                      │ kernel: float32[1,1,480,96]  │
│                    │                      │                              │
│                    │                      │ 46,176 (184.7 KB)            │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11/b2_2 │ float32[1,6,6,208]   │ bias: float32[208]           │
│                    │                      │ kernel: float32[3,3,96,208]  │
│                    │                      │                              │
│                    │                      │ 179,920 (719.7 KB)           │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11/b3_1 │ float32[1,6,6,16]    │ bias: float32[16]            │
│                    │                      │ kernel: float32[1,1,480,16]  │
│                    │                      │                              │
│                    │                      │ 7,696 (30.8 KB)              │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11/b3_2 │ float32[1,6,6,48]    │ bias: float32[48]            │
│                    │                      │ kernel: float32[5,5,16,48]   │
│                    │                      │                              │
│                    │                      │ 19,248 (77.0 KB)             │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11/b4_2 │ float32[1,6,6,64]    │ bias: float32[64]            │
│                    │                      │ kernel: float32[1,1,480,64]  │
│                    │                      │                              │
│                    │                      │ 30,784 (123.1 KB)            │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_11      │ float32[1,6,6,512]   │                              │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_12/b1_1 │ float32[1,6,6,160]   │ bias: float32[160]           │
│                    │                      │ kernel: float32[1,1,512,160] │
│                    │                      │                              │
│                    │                      │ 82,080 (328.3 KB)            │
├────────────────────┼──────────────────────┼──────────────────────────────┤
│ net/layers_12/b2_1 │ float32[1,6,6,112]   │ bias: float32[112]           │
│                    │                      │ kernel: float32[1,1,512,112] │
│                    │                      │                              │
│                    │                      │ 57,456 (229.8 KB)          

In [ ]:
model = GoogleNet(lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
# model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data, rngs={'params': random.PRNGKey(0), 'dropout': random.PRNGKey(1)})
# TODO: AssignSubModuleError: Submodule Conv must be defined in `setup()` or in a method wrapped in `@compact` (https://flax.readthedocs.io/en/latest/flax.errors.html#flax.errors.AssignSubModuleError)

## Discussion

A key feature of GoogLeNet is that it is actually *cheaper* to compute than its predecessors
while simultaneously providing improved accuracy. This marks the beginning of a much more deliberate
network design that trades off the cost of evaluating a network with a reduction in errors. It also marks the beginning of experimentation at a block level with network design hyperparameters, even though it was entirely manual at the time. This is largely due to the fact that deep learning frameworks in 2015 still lacked much of the design flexibility
that we now take for granted. Moreover, full network optimization is costly and at the time training on ImageNet still
proved computationally challenging.

Over the following sections we will encounter a number of design choices (e.g., batch normalization, residual connections, and channel grouping) that allow us to improve networks significantly. For now, you can be proud to have implemented what is arguably the first truly modern CNN.

## Exercises

1. GoogLeNet was so successful that it went through a number of iterations. There are several iterations
   of GoogLeNet that progressively improved speed and accuracy. Try to implement and run some of them.
   They include the following:
   1. Add a batch normalization layer :cite:`Ioffe.Szegedy.2015`, as described
      later in :numref:`sec_batch_norm`.
   1. Make adjustments to the Inception block (width, choice and order of convolutions), as described in
      :cite:`Szegedy.Vanhoucke.Ioffe.ea.2016`.
   1. Use label smoothing for model regularization, as described in
      :cite:`Szegedy.Vanhoucke.Ioffe.ea.2016`.
   1. Make further adjustments to the Inception block by adding residual connection
      :cite:`Szegedy.Ioffe.Vanhoucke.ea.2017`, as described later in
      :numref:`sec_resnet`.
1. What is the minimum image size for GoogLeNet to work?
1. Can you design a variant of GoogLeNet that works on Fashion-MNIST's native resolution of $28 \times 28$ pixels? How would you need to change the stem, the body, and the head of the network, if anything at all?
1. Compare the model parameter sizes of AlexNet, VGG, NiN, and GoogLeNet. How do the latter two network
   architectures significantly reduce the model parameter size?
1. Compare the amount of computation needed in GoogLeNet and AlexNet. How does this affect the design of an accelerator chip, e.g., in terms of memory size, amount of computation, and the benefit of specialized operations?


[Discussions](https://discuss.d2l.ai/t/82)
